## Spark Find Word Pairs

Find all the pairs of two consequent words where the first word is “narodnaya”. For each pair, count the number of occurrences in the Wikipedia dump. Print all the pairs with their count in a lexicographical order. Output format is “word_pair <tab> count”, for example:

red_apple	100500

crazy_zoo	42

Note that two words in a pair are concatenated with the underscore character, and the result is in the lowercase.

One motivation for counting these continuations is to get a better understanding of the language. Some words, like “the”, have a lot of continuations, while others, like “San”, have just a few (“San Francisco”, for example). One can build a language model with these statistics. If you are interested to learn more, search for “n-gram language model” in the Internet.

In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

In [2]:
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
wikipairs = wiki.flatMap(lambda x: [((x[i],x[i+1]),1) for i in range(0,len(x)-1)]).reduceByKey(lambda x,y:x+y)
wikinarod = wikipairs.filter(lambda x: x[0][0].lower() == "narodnaya").sortBy(lambda a: a[0][1])

In [3]:
result = wikinarod.collect()

In [22]:
for word in result:
    print "%s_%s\t%d" % (word[0][0].lower(),word[0][1].lower(),word[1])

narodnaya_gazeta	1
narodnaya_volya	9
